## S3 init

In [1]:
# Enter you Access and Secret keys. They're the ones that were displayed in the instructions.
s3_access_key_id = 'QHPHVRME5HNBBSXQ5FI7'
s3_secret_access_key = 'dzmMGm7FOjLztrTVCv0aCtJcwue9EQwfU715JZk2'

# Do not change this value, this is the internal location for the RGW
s3_endpoint_url = 'http://rook-ceph-rgw-s3a.openshift-storage.svc.cluster.local'

In [2]:
import boto3
import botocore

In [3]:
s3 = boto3.client('s3',
                endpoint_url = s3_endpoint_url,
                aws_access_key_id = s3_access_key_id,
                aws_secret_access_key = s3_secret_access_key,
                region_name = 'default',
                config=botocore.client.Config(signature_version = 's3'))

## Spark init

In [4]:
from pyspark.sql import SparkSession, SQLContext
import pyspark
from pyspark.context import SparkContext
import os
import socket

# create a spark session
spark_cluster_url = f"spark://{os.environ['SPARK_CLUSTER']}:7077"
spark = SparkSession.builder.master(spark_cluster_url).getOrCreate()

# test your spark connection
spark.range(5, numPartitions=5).rdd.map(lambda x: socket.gethostname()).distinct().collect()

['spark-cluster-gmoutier-40redhat-2ecom-w-mltjk',
 'spark-cluster-gmoutier-40redhat-2ecom-w-hjrf4']

In [5]:
print(f"Spark version = {spark.version}")

Spark version = 2.4.5


In [6]:
print(f"Hadoop version = {spark._jvm.org.apache.hadoop.util.VersionInfo.getVersion()}")

Hadoop version = 2.7.3


## Spark + S3

In [7]:
hadoopConf = spark.sparkContext._jsc.hadoopConfiguration()
hadoopConf.set("fs.s3a.endpoint", s3_endpoint_url)
hadoopConf.set("fs.s3a.access.key", s3_access_key_id)
hadoopConf.set("fs.s3a.secret.key", s3_secret_access_key)
hadoopConf.set("fs.s3a.path.style.access", "true")
hadoopConf.set("fs.s3a.connection.ssl.enabled", "false") # false if not https

In [8]:
print(hadoopConf.get("fs.s3a.endpoint"))
print(hadoopConf.get("fs.s3a.access.key"))
print(hadoopConf.get("fs.s3a.secret.key"))
print(hadoopConf.get("fs.s3a.path.style.access"))
print(hadoopConf.get("fs.s3a.connection.ssl.enabled"))

http://rook-ceph-rgw-s3a.openshift-storage.svc.cluster.local
QHPHVRME5HNBBSXQ5FI7
dzmMGm7FOjLztrTVCv0aCtJcwue9EQwfU715JZk2
true
false


In [15]:
hadoopConf.set("fs.s3a.signing-algorithm", "S3SignerType")

In [26]:
s3_bucket = 'DATA'
data = spark.read.csv('s3a://' + s3_bucket + '/sample_data.csv',sep=",", header=True)

df = data.toPandas()
df.head()

,Timestamp,Your Name,_c2,Primary Audience Engaged,"Key People (Presenters, Attendees, Organizers, etc)",Outcome,Full Notes,Email Address,Date,Notes,Lowlights,Learnings,Trip Region,Number of Days,Estimated Cost (USD),Product Mix
0,1/24/2017 16:46:47,John Doe,None,Customer,Customer 1,Successful,None,person@email.com,1/24/2017,Cras in purus eu magna vulputate luctus. Cum s...,In quis justo. Maecenas rhoncus aliquam lacus....,In quis justo. Maecenas rhoncus aliquam lacus....,NA,1,200,None
1,2/1/2017 12:09:48,John Doe,None,Event,Customer 2,Unsuccessful,None,person@email.com,1/16/2017,Nullam sit amet turpis elementum ligula vehicu...,Phasellus sit amet erat. Nulla tempus. Vivamus...,"Sed sagittis. Nam congue, risus semper porta v...",NA,5,200,None
2,2/3/2017 9:16:38,John Doe,None,Field,Customer 3,Unsuccessful,None,person@email.com,1/31/2017,"Mauris enim leo, rhoncus sed, vestibulum sit a...",Etiam vel augue. Vestibulum rutrum rutrum nequ...,In sagittis dui vel nisl. Duis ac nibh. Fusce ...,EMEA,3,200,None
3,2/5/2017 13:41:22,John Doe,None,Engineering,Customer 4,Successful,None,person@email.com,1/25/2017,Duis at velit eu est congue elementum. In hac ...,Mauris sit amet eros. Suspendisse accumsan tor...,Nullam porttitor lacus at turpis. Donec posuer...,EMEA,2,2000,None
4,2/5/2017 13:47:19,John Doe,None,Event,Customer 5,Successful,None,person@email.com,1/27/2017,"Sed sagittis. Nam congue, risus semper porta v...","Nulla neque libero, convallis eget, eleifend l...","Proin leo odio, porttitor id, consequat in, co...",EMEA,3,1000,None


## S3 Stuff

In [21]:
for bucket in s3.list_buckets()['Buckets']:
    print(bucket['Name'])

data_bucket


In [24]:
bucket_name='DATA'
for item in s3.list_objects_v2(Bucket=bucket_name)['Contents']:
    print(item['Key'])

sample_data.csv


In [20]:
s3.create_bucket(Bucket='DATA')

{'ResponseMetadata': {'RequestId': 'tx000000000000000001b50-0060075c17-aca5-s3a',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-request-id': 'tx000000000000000001b50-0060075c17-aca5-s3a',
   'content-length': '0',
   'date': 'Tue, 19 Jan 2021 22:24:23 GMT',
   'connection': 'Keep-Alive'},
  'RetryAttempts': 0}}

In [22]:
s3.upload_file("sample_data.csv", "DATA", "sample_data.csv")